<a href="https://colab.research.google.com/github/hnhoangdz/SIT_AI/blob/main/SIT_Project_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Import thư viện

In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# 2. Xử lí dữ liệu

In [9]:
df = pd.read_csv("https://raw.githubusercontent.com/hnhoangdz/SIT_AI/main/heart.csv")
df

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0
...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,M,TA,110,264,0,Normal,132,N,1.2,Flat,1
914,68,M,ASY,144,193,1,Normal,141,N,3.4,Flat,1
915,57,M,ASY,130,131,0,Normal,115,Y,1.2,Flat,1
916,57,F,ATA,130,236,0,LVH,174,N,0.0,Flat,1


In [10]:
df = pd.get_dummies(df, columns=['Sex', 'ChestPainType','RestingECG','ExerciseAngina','ST_Slope'])
df

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease,Sex_F,Sex_M,ChestPainType_ASY,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA,RestingECG_LVH,RestingECG_Normal,RestingECG_ST,ExerciseAngina_N,ExerciseAngina_Y,ST_Slope_Down,ST_Slope_Flat,ST_Slope_Up
0,40,140,289,0,172,0.0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,1
1,49,160,180,0,156,1.0,1,1,0,0,0,1,0,0,1,0,1,0,0,1,0
2,37,130,283,0,98,0.0,0,0,1,0,1,0,0,0,0,1,1,0,0,0,1
3,48,138,214,0,108,1.5,1,1,0,1,0,0,0,0,1,0,0,1,0,1,0
4,54,150,195,0,122,0.0,0,0,1,0,0,1,0,0,1,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,110,264,0,132,1.2,1,0,1,0,0,0,1,0,1,0,1,0,0,1,0
914,68,144,193,1,141,3.4,1,0,1,1,0,0,0,0,1,0,1,0,0,1,0
915,57,130,131,0,115,1.2,1,0,1,1,0,0,0,0,1,0,0,1,0,1,0
916,57,130,236,0,174,0.0,1,1,0,0,1,0,0,1,0,0,1,0,0,1,0


In [11]:
num_list= ["Age","RestingBP","Cholesterol","FastingBS","MaxHR","Oldpeak"]
for i in num_list:
  mean = df[i].mean()
  std = df[i].std()
  df[i] = (df[i]-mean)/std
df.head()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease,Sex_F,Sex_M,ChestPainType_ASY,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA,RestingECG_LVH,RestingECG_Normal,RestingECG_ST,ExerciseAngina_N,ExerciseAngina_Y,ST_Slope_Down,ST_Slope_Flat,ST_Slope_Up
0,-1.432359,0.410685,0.824621,-0.551041,1.382175,-0.831979,0,0,1,0,1,0,0,0,1,0,1,0,0,0,1
1,-0.478223,1.490940,-0.171867,-0.551041,0.753746,0.105606,1,1,0,0,0,1,0,0,1,0,1,0,0,1,0
2,-1.750404,-0.129442,0.769768,-0.551041,-1.524307,-0.831979,0,0,1,0,1,0,0,0,0,1,1,0,0,0,1
3,-0.584238,0.302660,0.138964,-0.551041,-1.131539,0.574398,1,1,0,1,0,0,0,0,1,0,0,1,0,1,0
4,0.051853,0.950812,-0.034736,-0.551041,-0.581664,-0.831979,0,0,1,0,0,1,0,0,1,0,1,0,0,0,1


In [12]:
Y = df['HeartDisease']
X = df.drop(columns=["HeartDisease"])

In [13]:
X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=123)
y_train = np.expand_dims(y_train,axis=1)
y_test = np.expand_dims(y_test,axis=1)
X_train, X_test, y_train, y_test = X_train.T, X_test.T, y_train.T, y_test.T

In [14]:
print("Training X: ",X_train.shape)
print("Training Y: ",y_train.shape)
print("Testing X: ",X_test.shape)
print("Testing Y: ",y_test.shape)

Training X:  (20, 734)
Training Y:  (1, 734)
Testing X:  (20, 184)
Testing Y:  (1, 184)


In [15]:
print(y_test)

[[0 1 1 0 1 0 1 0 0 1 1 1 1 0 1 0 0 0 0 1 1 1 0 1 1 0 0 1 0 1 0 1 0 0 0 0
  1 1 0 0 1 1 0 1 0 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 0 0 1 0 0 1 0 1
  1 1 0 1 0 1 0 0 0 0 0 1 0 1 1 0 1 1 0 0 0 1 1 1 0 0 1 0 1 1 0 0 0 0 1 0
  0 0 1 1 1 1 0 1 1 0 0 0 1 1 1 1 1 0 0 0 0 1 1 1 1 1 1 0 1 0 0 1 1 0 1 1
  0 1 0 1 1 0 0 0 0 1 1 0 1 1 0 1 1 0 1 0 1 1 0 1 0 1 0 0 1 1 1 1 1 1 0 1
  0 0 1 1]]


# 3. Build model

In [16]:
# Initilize parameters
def init_parameters(layers_dims):
  np.random.seed(3)
  L = len(layers_dims)
  parameters = {}
  for l in range(1,L):
    parameters["W" + str(l)] = np.random.randn(layers_dims[l],layers_dims[l-1])*np.sqrt(2/layers_dims[l-1])
    parameters["b" + str(l)] = np.zeros((layers_dims[l],1))
  return parameters

In [17]:
def sigmoid(Z):
  A = 1/(1+np.exp(-Z))
  return A,Z
def relu(Z):
  A = np.maximum(0,Z)
  return A,Z

In [18]:
def calc_ffw(W,A_prev,b,activation):
  if activation == "sigmoid":
    Z = np.dot(W,A_prev) + b
    A,Z = sigmoid(Z)
  else:
    Z = np.dot(W,A_prev) + b
    A,Z = relu(Z)
  linear_cache = (A_prev,W,b)
  activation_cache = Z
  cache = (linear_cache, activation_cache)
  return A,cache

In [19]:
def L_model_forward(X,parameters):
  L = len(parameters)//2
  caches = []
  A = X
  for l in range(1,L):
    A_prev = A
    A,cache = calc_ffw(parameters["W"+str(l)],A_prev,parameters["b"+str(l)],"relu")
    caches.append(cache)
  AL,cache = calc_ffw(parameters["W"+str(L)],A,parameters["b"+str(L)],"sigmoid")
  caches.append(cache)
  return AL, caches

In [30]:
def compute_cost(AL,Y,parameters, lamb):
  m = AL.shape[1]
  cross_entropy_cost = -1./m*(np.dot(Y,np.log(AL).T) + np.dot((1-Y),np.log(1-AL).T))
  sum_W = 0
  for l in range(1,len(parameters)//2+1):
    sum_W += np.sum(np.absolute(parameters["W"+str(l)]))
  L1_regularization_cost =  sum_W*lamb*1./(2*m)
  #L2_regularization_cost =  sum_W*lamb*1./(2*m)
  cost = np.squeeze(cross_entropy_cost + L1_regularization_cost) 
  #cost = np.squeeze(cross_entropy_cost + L2_regularization_cost)   
  return cost

In [31]:
def sigmoid_grad(dA,Z):
  A,Z = sigmoid(Z)
  dZ = dA*A*(1-A)
  return dZ
def relu_grad(dA,Z):
  A,Z = relu(Z)
  dZ = np.multiply(dA, np.int64(A > 0))
  return dZ

In [32]:
def linear_backward(dZ,cache, lamb):
  A_prev, W, b = cache
  m = A_prev.shape[1] 
  dW = 1./m*(np.dot(dZ,A_prev.T) + lamb*W/np.absolute(W))
  #dW = 1./m*np.dot(dZ,A_prev.T) + lamb*W/m
  db = 1./m*np.sum(dZ,axis=1,keepdims=True)
  dA_prev = np.dot(W.T,dZ)
  return dA_prev,dW,db

In [33]:
# compute backward with activation
def linear_activation_backward(dA,cache,activation,lamb):
  linear_cache, activation_cache = cache
  if activation == "relu":
    dZ = relu_grad(dA,activation_cache)
    dA_prev,dW,db = linear_backward(dZ,linear_cache,lamb)
  elif activation == "sigmoid":
    dZ = sigmoid_grad(dA,activation_cache)
    dA_prev,dW,db = linear_backward(dZ,linear_cache,lamb)
  return dA_prev,dW,db

In [34]:
# computer L_backward for NN
def L_model_backward(AL, Y, caches,lamb):
  grads = {}
  L = len(caches)
  Y = Y.reshape(AL.shape)
  dAL = np.divide(AL - Y, np.multiply(AL, 1 - AL))
  
  current_cache = caches[-1]
  grads["dA" + str(L-1)], grads["dW" + str(L)], grads["db" + str(L)] = linear_activation_backward(dAL,current_cache, activation = "sigmoid",lamb=lamb)

  for l in reversed(range(L-1)):
    current_cache = caches[l]
    dA_prev_temp, dW_temp, db_temp = linear_activation_backward(grads["dA" + str(l + 1)],  current_cache, activation = "relu", lamb=lamb)
    grads["dA" + str(l)] = dA_prev_temp
    grads["dW" + str(l + 1)] = dW_temp
    grads["db" + str(l + 1)] = db_temp
  return grads

In [35]:
def update_parameters(parameters, grads, lr):
  L = len(parameters)//2
  for l in range(L):
    parameters["W" + str(l+1)] = parameters["W" + str(l+1)]-lr*grads["dW" + str(l+1)]
    parameters["b" + str(l+1)] = parameters["b" + str(l+1)]-lr*grads["db" + str(l+1)]
  return parameters

In [36]:
def predict(X,Y,parameters):
  AL,cache =  L_model_forward(X,parameters)
  return np.round(AL)

In [37]:
def model(X,Y,X_test,y_test,num_iterations, learning_rate, lamb,print_cost=False):
  np.random.seed(3)
  input_size = X.shape[0]
  layer_dims = []
  layer_dims.append(input_size)
  layer_dims.append(16)
  #layer_dims.append(8)
  layer_dims.append(4)
  layer_dims.append(1)
  parameters = init_parameters(layer_dims)
  #return parameters
  costs = []
  for i in range(num_iterations):
    AL, caches = L_model_forward(X,parameters)
    cost = compute_cost(AL, Y,parameters,lamb=lamb)
    costs.append(cost)
    grads = L_model_backward(AL,Y,caches,lamb=lamb)
    parameters = update_parameters(parameters,grads,learning_rate)
    if print_cost and i % 1000 == 0:
      print ("Cost after iteration %i: %f" %(i, cost))
  Y_train_pred = predict(X,Y,parameters)
  Y_test_pred = predict(X_test,y_test,parameters)
  print("train accuracy: {} %".format(100 - np.mean(np.abs(Y_train_pred - Y)) * 100))
  print("test accuracy: {} %".format(100 - np.mean(np.abs(Y_test_pred - y_test)) * 100))
  return parameters,costs

In [38]:
parameters,costs = model(X_train,y_train,X_test,y_test,15000,0.01,0.9,True)

Cost after iteration 0: 1.126433
Cost after iteration 1000: 0.463594
Cost after iteration 2000: 0.391638
Cost after iteration 3000: 0.366346
Cost after iteration 4000: 0.348872
Cost after iteration 5000: 0.336391
Cost after iteration 6000: 0.326059
Cost after iteration 7000: 0.315574
Cost after iteration 8000: 0.307402
Cost after iteration 9000: 0.298998
Cost after iteration 10000: 0.291180
Cost after iteration 11000: 0.283212
Cost after iteration 12000: 0.276854
Cost after iteration 13000: 0.269612
Cost after iteration 14000: 0.262530
train accuracy: 92.50681198910081 %
test accuracy: 89.13043478260869 %


In [29]:
15000,0.01,0.9
15000,0.01,0.8

(15000, 0.01, 0.8)